In [ ]:
# CSV 파일 전처리
# 엑셀로 만든 CSV 파일 인코딩을 엑셀에서 UTF-8로 저장해도 UTF-8 이나 EUC-KR 로 인코딩할 때 에러 발생
# CP949로 인코딩 성공
import pandas as pd


columns =['일자', '품목구분', '품목코드', '품목명', '규격', '수량', '단위', '단가', '공급가', '합계', '거래처코드', '거래처', '거래처그룹1', '거래처그룹2', '담당자', '개설연도', '지역구분', '인센가입', '품목그룹1', '품목그룹2', '인센품목']
dType = {
    '일자':str, 
    '품목구분':str, 
    '품목코드':str, 
    '품목명':str, 
    '규격':str, 
    '수량':str, 
    '단위':str, 
    '단가':str, 
    '공급가':str, 
    '합계':str, 
    '거래처코드':str, 
    '거래처':str, 
    '거래처그룹1':str, 
    '거래처그룹2':str, 
    '담당자':str, 
    '개설연도':str, 
    '지역구분':str, 
    '인센가입':str, 
    '품목그룹1':str, 
    '품목그룹2':str, 
    '인센품목':str
}

def clean_file(fileName, save=False, replace=False):
    if replace:
        save = True
    [fname, ftype] = str(fileName).split('.')
    
    try:
        if ftype == 'csv':
            df = pd.read_csv(fileName, header=1, index_col=False, names=columns, skipfooter=3, engine='python', dtype=dType)
        elif ftype == 'xlsx':
            df = pd.read_excel(fileName, header=1, index_col=False, names=columns, skipfooter=2, dtype=dType)
        df = check(df)

        if not replace:
            fileName = fname+'_cleaned.'+ftype

        if save:
            if ftype == 'csv':
                df.to_csv(fileName, index=False)
            elif ftype == 'xlsx':
                df.to_excel(fileName, index=False)
        return df

    except UnicodeDecodeError:
        print('Error')
        df = pd.read_csv(fileName, encoding='CP949')
        df = check(df)
        return df

    except Exception as ex:
        print('%s' % (ex))


def check(df):
    colList = list(df.columns)
    for col in colList:
        if "Unnamed:" in col:  # 이름이 없는 칼럼 삭제
            df = df.drop(col, axis=1)
            colList = list(df.columns)
    newcolList = []
    for col in colList:
        if "\t" in col:
            new = str(col).replace('\t', '')
        else:
            new = col
        newcolList.append(new)
    df.columns = newcolList
    df.replace('\t', '', regex=True, inplace=True)
    return df



In [ ]:
import pandas as pd

# 파일 정보 추출
footerRowN = 1
headerRow = 2
path = 'D:/마케팅 공용/원자료/품목등록/'
fName = '품목등록(20230619조회).xlsx'

# 칼럼명 추출
cstr = '품목구분	No.	품목코드	품목명	규격명	단위	검색창내용	생산공정	생산라인	구매처명	최소구매	조달기간	도번	품목그룹1	품목그룹2	재고관리	표준단가	사용	제품유형'
columns=cstr.split('\t')
dType = dict.fromkeys(columns, str) #데이터타입을 string으로 인식하여 읽어오기 위해

In [ ]:
df = pd.read_excel(path+fName, names=columns, index_col=False, header=headerRow, skipfooter=footerRowN, dtype=dType)
df = check(df)
df.to_csv('test.csv', index=False, encoding='utf-8-sig')

In [ ]:
path = 'D:/마케팅 공용/원자료/CSV/'
fname = '2006-2022본사영남판매현황(영업데이터분석).csv'
cstr = '일자	품목구분	품목코드	품목명	규격	수량	단위	단가	공급가	합계	거래처코드	거래처	거래처그룹1	거래처그룹2	담당자	개설연도	지역구분	인센가입	품목그룹1	품목그룹2	인센품목'
columns = cstr.split('\t')
dType = dict.fromkeys(columns, str)

sales_all = pd.read_csv(path+fname, names=columns, header=0, dtype=dType)
sales_all

In [ ]:
ndf = pd.DataFrame(df, columns=['품목코드', '제품유형'])
m = pd.merge(sales_all, ndf, how='left', on='품목코드')
m.to_csv('test.csv', index=False, encoding='utf-8-sig')
